In [162]:
import os
import sys
import random
sys.path.append('../')

import numpy as np

from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from rocket_rag.transform import *
from rocket_rag.node_indexing import *

In [19]:
cfg = json.load(open("../config/configs.json"))

INSTANCES_DIR = '../data/instances/'
INFERENCE_DIR = '../data/inference/'
STATES = ['normal', 
            'backlash1', 'backlash2',
            'lackLubrication1', 'lackLubrication2',
            'spalling1', 'spalling2', 'spalling3', 'spalling4', 'spalling5', 'spalling6', 'spalling7', 'spalling8']
LOADS= ['20kg', '40kg', '-40kg']
DOC_DIR = '../docs'

In [96]:
cur_knn = KNeighborsClassifier(n_neighbors=1)
pos_knn = KNeighborsClassifier(n_neighbors=1)
cur_ridge = RidgeClassifier()
pos_ridge = RidgeClassifier()

In [97]:
ts_transform = TimeSeriesTransform(cfg)
ts_node_indexer = TimeSeriesNodeIndexer()

In [49]:
curs = ts_node_indexer.load_nodes(f'../store/ts_indexing/current_nodes_{load}.pkl')
poss = ts_node_indexer.load_nodes(f'../store/ts_indexing/position_error_nodes_{load}.pkl')

2024-09-26 16:51:48.916 | DEBUG    | rocket_rag.node_indexing:load_nodes:59 - Loading all nodes...
2024-09-26 16:51:49.884 | DEBUG    | rocket_rag.node_indexing:load_nodes:63 - All nodes are loaded.
2024-09-26 16:51:49.885 | DEBUG    | rocket_rag.node_indexing:load_nodes:59 - Loading all nodes...
2024-09-26 16:51:50.836 | DEBUG    | rocket_rag.node_indexing:load_nodes:63 - All nodes are loaded.


In [129]:
load = '20kg'
filenames = [os.listdir(os.path.join(INFERENCE_DIR, load, state)) for state in STATES]
filenames = [filename for sublist in filenames for filename in sublist]

In [130]:
rand_filename = random.choice(filenames)
load_num = load[:2]
state = re.match(fr'(.*)_{load_num}', rand_filename).group(1)
rand_ts_df = pd.read_csv(os.path.join(INFERENCE_DIR, load, state, rand_filename))
print(rand_filename)

spalling5_20_1_3.csv


In [131]:
rand_cur = ts_transform.smoothing(ts_df=rand_ts_df, field='current')
rand_pos = ts_transform.smoothing(ts_df=rand_ts_df, field='position_error')

### ROCKET time domain rocket feature

In [98]:
curs_rocket = [c.rocket for c in curs]
poss_rocket = [p.rocket for p in poss]
curs_labels = [c.id_ for c in curs]
poss_labels = [p.id_ for p in poss]

In [99]:
cur_knn.fit(curs_rocket, curs_labels)
pos_knn.fit(poss_rocket, poss_labels)
cur_ridge.fit(curs_rocket, curs_labels)
pos_ridge.fit(poss_rocket, poss_labels)

RidgeClassifier()

In [132]:
rand_cur_rocket = ts_transform.get_rocket(rand_cur)
rand_pos_rocket = ts_transform.get_rocket(rand_pos)

In [133]:
print(f'KNN for current signal: {cur_knn.predict([rand_cur_rocket])}')
print(f'KNN for position error: {pos_knn.predict([rand_pos_rocket])}')
print(f'Ridge for current signal: {cur_ridge.predict([rand_cur_rocket])}')
print(f'Ridge for position error: {pos_ridge.predict([rand_pos_rocket])}')

KNN for current signal: ['spalling4']
KNN for position error: ['spalling5']
Ridge for current signal: ['spalling5']
Ridge for position error: ['spalling5']


### FFT Frequency domain

In [157]:
curs_fft_magn = [c.fft[0] for c in curs]
poss_fft_magn = [p.fft[0] for p in poss]

In [163]:
def complex_to_mag_phase(X):
    magnitude = np.abs(X)
    phase = np.angle(X)
    return np.column_stack((magnitude, phase))

In [169]:
curs_fft_magn[0]

array([ 2.44582847e+02+0.00000000e+00j,  3.02796733e+01+1.54402108e+01j,
       -2.51454942e+01+1.31166551e+01j,  9.86187046e+00-1.85789096e+00j,
       -6.58799850e+00+1.08799763e+01j, -1.34853282e+00+2.04787423e+00j,
        4.35700444e+00+3.94907590e+00j, -3.46403784e+00+4.94122254e+00j,
        1.78692685e+00-4.09969326e-01j,  3.88724670e+00+4.25510696e+00j,
       -3.81825607e+00+7.29115422e-02j,  4.55267615e+00-2.51746146e-01j,
       -2.01292690e+00+1.07136593e+00j, -7.86027416e-01+6.91839698e-01j,
        2.12983420e+00-1.83282086e+00j, -2.78475185e+00+2.53519572e+00j,
        5.36075864e-01-3.12298488e+00j, -2.18729749e+00+1.10706324e+00j,
       -2.32331221e+00+1.68094291e+00j, -5.36479389e-01-1.50246742e+00j,
       -2.08769570e+00+2.98080650e+00j, -6.91060254e-01+2.74306540e-01j,
       -9.95511212e-01-3.55892005e-01j, -6.80390246e-01+2.75084858e+00j,
       -2.15640342e+00-5.09125919e-01j,  8.17295057e-02+1.96902288e+00j,
       -1.90544108e+00+1.03924230e+00j, -8.74958704

In [171]:
complex_to_mag_phase(curs_fft_magn)

324

In [160]:
cur_fft_knn = KNeighborsClassifier(n_neighbors=1)
pos_fft_knn = KNeighborsClassifier(n_neighbors=1)
cur_fft_ridge = RidgeClassifier()
pos_fft_ridge = RidgeClassifier()

In [161]:
cur_fft_knn.fit(curs_fft_magn, curs_labels)
pos_fft_knn.fit(poss_fft_magn, poss_labels)
cur_fft_ridge.fit(curs_fft_magn, curs_labels)
pos_fft_ridge.fit(poss_fft_magn, poss_labels)

ValueError: Complex data not supported
[[ 2.44582847e+02+0.00000000e+00j  3.02796733e+01+1.54402108e+01j
  -2.51454942e+01+1.31166551e+01j ... -3.03870891e-01+4.03788757e-02j
  -3.63206858e-01+2.49994365e-02j -3.22439190e-01+9.26105963e-02j]
 [ 2.46372755e+02+0.00000000e+00j  2.95364283e+01+1.45645371e+01j
  -2.40387815e+01+1.31351728e+01j ... -3.81391694e-01-3.53821119e-02j
  -4.06745893e-01+4.79553780e-02j -3.19476033e-01-1.98720115e-02j]
 [ 2.45767426e+02+0.00000000e+00j  3.03879671e+01+1.44911542e+01j
  -2.47449516e+01+1.31210553e+01j ... -3.14534890e-01+6.18997828e-02j
  -2.09378418e-01+3.86114374e-02j -3.81052293e-01+9.46564656e-03j]
 ...
 [ 2.34281575e+02+0.00000000e+00j  3.00157055e+01+1.36757769e+01j
  -2.44397140e+01+1.35746893e+01j ... -2.77000426e-01-2.58914801e-03j
  -3.72493424e-01+4.69247293e-02j -1.96189054e-01+2.32139956e-02j]
 [ 2.33387133e+02+0.00000000e+00j  2.84635564e+01+1.45144083e+01j
  -2.34805095e+01+1.29846859e+01j ... -2.71312437e-01-8.88719833e-02j
  -3.180